Import libraries and modules

In [1]:
import re
from collections import Counter


from urlextract import URLExtract
import nltk
import numpy as np
import pandas as pd
import random
import torch

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit

Simple datamodel class

In [2]:
class Document:
    def __init__(self, raw_text, label=None, tokens=None):
        self._raw_text = raw_text
        self._label = label
        self._tokens = None
        if tokens is not None:
            self._tokens = list(tokens)
    
    @property
    def raw_text(self):
        return self._raw_text
    
    @property
    def label(self):
        return self._label
    
    @property
    def tokens(self):
        return list(self._tokens)
    
    def tokenized(self, tokenizer):
        return Document(self._raw_text, self._label, tokenizer.tokenize(self._raw_text))
    
    def __repr__(self):
        return f"{self._label}\t{self._raw_text}\t{self._tokens if self._tokens is not None else ''}\n"

Useful function to get iterator over docs labels

In [3]:
def docs_labels(docs):
    return (d.label for d in docs)

Load raw data in our datamodel

In [4]:
def load_dataset(path):
    df = pd.read_csv(path, delimiter=',', encoding='latin-1')
    return [Document(text, label) for text, label in zip(df.v2, df.v1)]

In [5]:
docs = load_dataset("spam.csv")

Check data

In [6]:
docs[:5]

[ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...	,
 ham	Ok lar... Joking wif u oni...	,
 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's	,
 ham	U dun say so early hor... U c already then say...	,
 ham	Nah I don't think he goes to usf, he lives around here though	]

In [7]:
Counter(docs_labels(docs))

Counter({'ham': 4825, 'spam': 747})

Let's check SMS that we are interested to find

In [7]:
for i, doc in enumerate(d for d in docs if d.label == "spam"):
    if i >= 25:
        break
    print(doc.raw_text)
    print()

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv

WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info

URGENT! You have won a 1 week FREE membership in our å£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18

XXXMobileMovieClub: To use your credit, click the WAP link in the next txt messag

So many digits and urls. Let's use twitter-based NLTK tokenizer and do some preprocessing: change all digits to one 0 and change all urls to '<href>' token

In [8]:
class TwitterNLTKTokenizer:
    DIGITS_RE = re.compile(r'[0-9]+')
    
    def __init__(self):
        nltk.download('punkt')
        self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TweetTokenizer(preserve_case=False, reduce_len=True)
        self.url_extractor = URLExtract()
        
    def tokenize(self, text):
        text = self.DIGITS_RE.sub('0', text)
        tokens = []
        for sent in self.sent_detector.tokenize(text):
            urls = self.url_extractor.find_urls(sent, only_unique=True)
            for url in urls:
                sent = sent.replace(url, "<href>")
            tokens.extend(self.tokenizer.tokenize(sent))            
                
        return tokens

Tokenize whole dataset

In [9]:
tokenizer = TwitterNLTKTokenizer()
docs = [d.tokenized(tokenizer) for d in docs]

[nltk_data] Downloading package punkt to /home/vladislav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Check tokenization

In [10]:
for d in docs[:5]:
    print(d)

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...	['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']

ham	Ok lar... Joking wif u oni...	['ok', 'lar', '...', 'joking', 'wif', 'u', 'oni', '...']

spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's	['free', 'entry', 'in', '0', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '0st', 'may', '0', '.', 'text', 'fa', 'to', '0', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 't', '&', "c's", 'apply', '0over0', "'", 's']

ham	U dun say so early hor... U c already then say...	['u', 'dun', 'say', 'so', 'early', 'hor', '...', 'u', 'c', 'already', 'then', 'say', '...']

ham	Nah I don't think he goes to us

Let's do simple TFIDF features for our small texts, may be we can find some keywords through them

In [11]:
class TfIDFFeatureExtractor:
    def __init__(self, docs):
        self._vectorizer = TfidfVectorizer(
            token_pattern=None,
            tokenizer=self._tokenize_doc,
            stop_words='english', min_df=0.001,
            preprocessor=self._identity
        )
        self._vectorizer.fit(docs)
    
    @property
    def vector_length(self):
        return len(self._vectorizer.vocabulary_)
    
    def vectorize_docs(self, docs):
        return [f.toarray()[0] for f in self._vectorizer.transform(docs)]
        
    @staticmethod
    def _tokenize_doc(doc):
        return doc.tokens
    
    @staticmethod
    def _identity(x):
        return x

Function to iterate over dataset, generate features and collect samples into batches

In [12]:
from itertools import repeat

def batcherize(docs, feature_extractor, batch_size, shuffle=True, labels=True):
    if shuffle:
        docs = list(docs)
        random.shuffle(docs)
    
    samples = feature_extractor.vectorize_docs(docs)
    labels = docs_labels(docs) if labels else repeat(None)
    
    samples_queue, labels_queue = [], []
    
    for sample, label in zip(samples, labels):
        if len(samples_queue) == batch_size:
            yield samples_queue, labels_queue
            samples_queue, labels_queue = [], []
        
        samples_queue.append(sample)
        labels_queue.append(1 if label == "spam" else 0)
    
    if samples_queue:
        yield samples_queue, labels_queue

Let's evaluate with F1 because we have imbalanced classes

In [13]:
def evaluate(predicted_labels, gold_labels):
    def to_int(labels, positive_label="spam"):
        return [int(l == positive_label) for l in labels]
    return f1_score(to_int(gold_labels), to_int(predicted_labels))

Simple pytorch multilayered perceptron network for binary classification with sigmoid output

In [14]:
class BinaryClassificationMLP(torch.nn.Module):
    def __init__(self, input_size: int, hidden_sizes: list, activation=torch.nn.LeakyReLU):
        super().__init__()
        
        self._activation = activation()
        self._hidden_layers = []
                
        prev_size = input_size
        for size in hidden_sizes:
            layer = torch.nn.Linear(prev_size, size)
            self._hidden_layers.append(layer)
            prev_size = size
        
        self._out_layer = torch.nn.Linear(prev_size, 1)
        self._out_sigmoid = torch.nn.Sigmoid()
    
    def forward(self, sample):
        for layer in self._hidden_layers:
            sample = self._activation(layer(sample))
        
        return self._out_sigmoid(self._out_layer(sample))            

We need docs to train and to evaluate, let's split them. Also we are accurate with class imbalance

In [15]:
def train_dev_test_split(docs, test_ratio, dev_ratio, random_seed=100):
    def index_docs(docs, indexes):
        return [docs[idx] for idx in indexes]
        
    train, dev_test = iter(next(
        StratifiedShuffleSplit(1, test_ratio + dev_ratio, random_state=random_seed).split(
            docs, list(docs_labels(docs)))))
    
    train_docs = index_docs(docs, train)
    dev_test_docs = index_docs(docs, dev_test)
    
    dev, test = iter(next(
        StratifiedShuffleSplit(1, test_ratio / (dev_ratio + test_ratio), random_state=random_seed).split(
            dev_test_docs, list(docs_labels(dev_test_docs)))))
    
    return train_docs, index_docs(dev_test_docs, dev), index_docs(dev_test_docs, test)

In [16]:
train, dev, test = train_dev_test_split(docs, 0.15, 0.15)

Function to get predictions from our model

In [17]:
def predict(model, dev, batcher_factory):
    predictions = []
    
    with torch.no_grad():
        for samples, _ in batcher_factory(dev, False, False):
            outputs = model(torch.tensor(samples, dtype=torch.float32))
            predictions.extend(l[0] for l in outputs.numpy())
    return ["spam" if p > 0.5 else "ham" for p in predictions]

Function to train model, returns max score we got on dev set

In [18]:
def train_model(epoch_num, train, dev, batcher_factory, model, optimizer, criterion):
    max_dev_score = 0
    for epoch in range(epoch_num):
        running_loss = 0.0
        for i, (samples, labels) in enumerate(batcher_factory(train, True, True)):
            optimizer.zero_grad()
            outputs = model(torch.tensor(samples, dtype=torch.float32))
            loss = criterion(outputs, torch.tensor(labels, dtype=torch.float32).unsqueeze_(-1))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 300 == 299:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
        print(f"train score = {evaluate(predict(model, train, batcher_factory), docs_labels(train))}")
        dev_score = evaluate(predict(model, dev, batcher_factory), docs_labels(dev))
        print(f"dev score = {dev_score}")
        max_dev_score = max(dev_score, max_dev_score)
    
    return max_dev_score

Factory for our batcher with feature extractor. Need to decompose batcher and feature extractor but not enough motivation to do it in pet project :)

In [19]:
def get_batcher_factory(feature_extractor, batch_size):
    return lambda docs, shuffle, labels: batcherize(docs, feature_extractor, batch_size, shuffle, labels)

Experiment pipeline. Evaluate model with 5 seeds and make results reproducible by setting seeds everywhere

In [20]:
seed_num = 5

epoch_num = 20
hidden_sizes = [200]
learning_rate = 0.03
batch_size = 6
dev_results = []

for seed in range(100, 100*seed_num, 100):
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    
    feature_extractor = TfIDFFeatureExtractor(train)
    model = BinaryClassificationMLP(feature_extractor.vector_length, hidden_sizes, torch.nn.Tanh)
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    max_dev_score = train_model(
        epoch_num, train, dev, get_batcher_factory(feature_extractor, batch_size), model, optimizer, criterion)
    dev_results.append(max_dev_score)

print()
print(f'mean_score = {np.mean(dev_results)}, std = {np.std(dev_results)}')

[1,   300] loss: 0.048
[1,   600] loss: 0.038
train score = 0.6169665809768637
dev score = 0.6628571428571429
[2,   300] loss: 0.028
[2,   600] loss: 0.025
train score = 0.7671541057367829
dev score = 0.792079207920792
[3,   300] loss: 0.024
[3,   600] loss: 0.021
train score = 0.7951807228915663
dev score = 0.8155339805825244
[4,   300] loss: 0.021
[4,   600] loss: 0.020
train score = 0.801317233809001
dev score = 0.8195121951219512
[5,   300] loss: 0.019
[5,   600] loss: 0.019
train score = 0.8180839612486545
dev score = 0.8285714285714286
[6,   300] loss: 0.018
[6,   600] loss: 0.018
train score = 0.7968923418423973
dev score = 0.8078817733990148
[7,   300] loss: 0.015
[7,   600] loss: 0.017
train score = 0.8392484342379959
dev score = 0.839622641509434
[8,   300] loss: 0.016
[8,   600] loss: 0.016
train score = 0.8841285296981499
dev score = 0.8558558558558559
[9,   300] loss: 0.016
[9,   600] loss: 0.015
train score = 0.8467153284671532
dev score = 0.8544600938967137
[10,   300] l

[15,   600] loss: 0.013
train score = 0.8781985670419653
dev score = 0.8598130841121495
[16,   300] loss: 0.014
[16,   600] loss: 0.015
train score = 0.8843813387423936
dev score = 0.8598130841121495
[17,   300] loss: 0.014
[17,   600] loss: 0.015
train score = 0.889108910891089
dev score = 0.8454545454545456
[18,   300] loss: 0.013
[18,   600] loss: 0.015
train score = 0.8875739644970414
dev score = 0.8506787330316743
[19,   300] loss: 0.015
[19,   600] loss: 0.014
train score = 0.8811475409836066
dev score = 0.8598130841121495
[20,   300] loss: 0.013
[20,   600] loss: 0.014
train score = 0.888
dev score = 0.8623853211009174

mean_score = 0.8693969332037392, std = 0.012911986204209908


0.86 F1 is good result! Pipeline works even without complex features and complex classifiers. We can go further with hyperparameter search, deeper networks and final test evaluation but it's just routine